# Libraries and modules


In [1]:
import pandas as pd 
import os 
import datetime
from FR.frontpage import scrapeFrontPage, regexFrontData
from FR.generate_url import returnUrlHousingtoScrap 
from FR.tools import savedata as sd
from FR.directories import master_directory
#Visual de data (permite ver todo un df)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# __ENVIRONMENT__

In [2]:
# Variables 
month = 'FEB'       #Update each month
year = '2024'         #Update if the year changes

# Constants  
OPERATION = 'Sale'
WEB = 'FR'

#Master directory
master_dir = master_directory()
master_dir = f'{master_dir}/{WEB}'


#Directories to export files 
result_dir = f'{master_dir}/FRONTPAGE/JOIN'
monitoring_dir = f'{master_dir}/FRONTPAGE/MONITORING/{year}/{month}'

#Export files 
###############################################################################################################
#This files are going to be joined for create the master file, which contains all the data for the fron page
fl_pre = f'{WEB}_join_{OPERATION}_{month}{year}_'

fl_master = f'{WEB}_{month}{year}_{OPERATION}_Front'


# Generate links 

In [3]:
df_enlaces = returnUrlHousingtoScrap('Sales')

Enlaces generados: 370


# Split data 

In [4]:
# Divide the df in 24 observations for don't be catch in the scraping 
num_dfs = len(df_enlaces) // 24 + (1 if len(df_enlaces) % 24 != 0 else 0)

dfs = {f'{fl_pre}{i}': df_enlaces.iloc[i:i+24] for i in range(0, len(df_enlaces), 24)}

print(f'You have created {num_dfs} DataFrames from df_enlaces:')
for key in dfs:
    print(f'{key}')

Se crearon 16 DataFrames para df_enlaces:
FR_join_Sale_FEB2024_0
FR_join_Sale_FEB2024_24
FR_join_Sale_FEB2024_48
FR_join_Sale_FEB2024_72
FR_join_Sale_FEB2024_96
FR_join_Sale_FEB2024_120
FR_join_Sale_FEB2024_144
FR_join_Sale_FEB2024_168
FR_join_Sale_FEB2024_192
FR_join_Sale_FEB2024_216
FR_join_Sale_FEB2024_240
FR_join_Sale_FEB2024_264
FR_join_Sale_FEB2024_288
FR_join_Sale_FEB2024_312
FR_join_Sale_FEB2024_336
FR_join_Sale_FEB2024_360


# __Scraping__

In [5]:
try:
    start_time = datetime.datetime.now()
    print(f"The process began at {start_time.strftime('%H:%M')} of {start_time.strftime('%d/%m/%Y')}")
    
    for df_key, df_value in dfs.items():
        print(f"Scraping of {df_key}")
        global resultado_df
        # Create en empty df 
        resultado_df = pd.DataFrame()
        contador_actual = 0
        # Create a list for storage the results of each url 
        resultados = []
        # Iterate between the list of urls 
        for index, row in df_value.iterrows():
            contador_actual += 1
            print(f"Enlace {contador_actual} de {len(df_value)}")
            
            link = row["Enlace"]
            df_resultado = scrapeFrontPage(link) 
            resultados.append(df_resultado)

        # Join the results in a df 
        resultado_df = pd.concat(resultados, ignore_index=True)
        
        # Save the data in the directory of results
        sd(resultado_df, f'{result_dir}/{df_key}')

        print(f"Scraping completado para {df_key}.")
        print("--------------------------------------------------")
    # End the scraping: 
    end_time = datetime.datetime.now()
    print(f"The process have ended at {end_time.strftime('%H:%M')} of {end_time.strftime('%d/%m/%Y')}")
    
except Exception as e:
    print(f"Se produjo un error en el scraping: {e}")

El proceso de scraping ha comenzado a las 07:48 del 01/02/2024
Realizando scraping en FR_join_Sale_FEB2024_0
Enlace 1 de 24
Enlace 2 de 24
Enlace 3 de 24
Enlace 4 de 24
Enlace 5 de 24
Enlace 6 de 24
Enlace 7 de 24
Enlace 8 de 24
Enlace 9 de 24
Enlace 10 de 24
Enlace 11 de 24
Enlace 12 de 24
Enlace 13 de 24
Enlace 14 de 24
Enlace 15 de 24
Enlace 16 de 24
Enlace 17 de 24
Enlace 18 de 24
Enlace 19 de 24
Enlace 20 de 24
Enlace 21 de 24
Enlace 22 de 24
Enlace 23 de 24
Enlace 24 de 24
Scraping completado para FR_join_Sale_FEB2024_0.
--------------------------------------------------
Realizando scraping en FR_join_Sale_FEB2024_24
Enlace 1 de 24
Enlace 2 de 24
Enlace 3 de 24
Enlace 4 de 24
Enlace 5 de 24
Enlace 6 de 24
Enlace 7 de 24
Enlace 8 de 24
Enlace 9 de 24
Enlace 10 de 24
Enlace 11 de 24
Enlace 12 de 24
Enlace 13 de 24
Enlace 14 de 24
Enlace 15 de 24
Enlace 16 de 24
Enlace 17 de 24
Enlace 18 de 24
Enlace 19 de 24
Enlace 20 de 24
Enlace 21 de 24
Enlace 22 de 24
Enlace 23 de 24
Enlace 24 

Scraping completado para FR_join_Sale_FEB2024_336.
--------------------------------------------------
Realizando scraping en FR_join_Sale_FEB2024_360
Enlace 1 de 10
Enlace 2 de 10
Enlace 3 de 10
Enlace 4 de 10
Enlace 5 de 10
Enlace 6 de 10
Enlace 7 de 10
Enlace 8 de 10
Enlace 9 de 10
Enlace 10 de 10
Scraping completado para FR_join_Sale_FEB2024_360.
--------------------------------------------------
El proceso de scraping ha terminado a las 16:08 del 01/02/2024


## Join data 

In [6]:
# Obtain the list of names of files you saved in the result diectory
# This is going to take into account the name of each file 
archivos_csv = [archivo for archivo in os.listdir(result_dir) if 
                archivo.endswith('.csv') and archivo.startswith(fl_pre)]

# Save the dataframes in this empty list 
dataframes = []

# Reading each CSV file 
for archivo in archivos_csv:
    ruta_completa = os.path.join(result_dir, archivo)
    df = pd.read_csv(ruta_completa, sep=';', encoding='utf-16')
    dataframes.append(df)
    

# Join all the files
portadas = pd.concat(dataframes, ignore_index=True)

# Clean the data, take just the principal information...  
# ... for each variable using regexFrontData function 
exportable = regexFrontData(portadas)
print(len(exportable))

251365


## Export the master file

In [7]:
sd(exportable, f'{monitoring_dir}/{fl_master}')